In [2]:
from janome.tokenizer import Tokenizer
from gensim.models import word2vec
import re
import requests
from bs4 import BeautifulSoup

In [13]:
base_url = "https://www.uta-net.com"
target_url = 'https://www.uta-net.com/search/?Aselect=1&Bselect=4&Keyword=%E4%B8%AD%E6%A3%AE%E6%98%8E%E8%8F%9C&sort=4'
music_num = 200

r = requests.get(target_url)

soup = BeautifulSoup(r.text, "html.parser")
url_list = []
#曲一覧から各曲のURLを取り出してリストに入れる
for i in range(music_num):
   href = soup.find_all("td", attrs={"class": "side td1"})[i].contents[0].get("href")
   url_list.append(href)         

kashi = ""
#曲ごとにRequestを送り歌詞を抽出する
for i in range(music_num):
   target_url = base_url + url_list[i]
   r = requests.get(target_url)
   soup = BeautifulSoup(r.text, "html.parser")

   for string in soup.find_all("div", attrs={"id": "kashi_area"})[0].strings:
       kashi += string

with open('./kashi_txt/akina_kashi_200', mode = 'w', encoding = 'utf-8') as fw:
   fw.write(kashi)

In [14]:
import re
# 英数字の削除
kashi = re.sub("[a-xA-Z0-9_]","",kashi)
# 記号の削除
kashi = re.sub("[!-/:-@[-`{-~]","",kashi)
# 空白・改行の削除
kashi = re.sub(u'\n\n', '\n', kashi)
kashi = re.sub(u'\r', '', kashi)

In [15]:
def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(text)
    word = []
    stop_word = create_stop_word()
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
        if part_of_speech == "名詞":
            if not token.surface in stop_word:
                word.append(token.surface)        
        if part_of_speech == "動詞":
            if not token.base_form in stop_word:
                word.append(token.base_form)
        if part_of_speech == "形容詞":
            if not token.base_form in stop_word:
                word.append(token.base_form)        
        if part_of_speech == "形容動詞":        
            if not token.base_form in stop_word:
                word.append(token.base_form)

    return word

In [16]:
def create_stop_word():
    target_url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    r =requests.get(target_url)
    soup=BeautifulSoup(r.text, "html.parser")
    stop_word=str(soup).split()
    return stop_word

In [17]:
sentence = [tokenize(kashi)]
model = word2vec.Word2Vec(sentence, size=100, min_count=4, window=4, iter=50)
print(model.wv.most_similar(positive=[u"人生"], topn=10))

[('ゲーム', 0.9926275610923767), ('合う', 0.9812482595443726), ('ドラマ', 0.979516863822937), ('つくる', 0.9782220721244812), ('辿る', 0.973213255405426), ('今度', 0.972602128982544), ('なくす', 0.9723961353302002), ('時代', 0.9709115624427795), ('こわれる', 0.9708400368690491), ('ふれる', 0.9702985286712646)]
